In [1]:
import tensorflow as tf
from f3dasm.functions.pybenchfunction import Sphere

import autograd
import autograd.core
import autograd.numpy as np
from autograd import elementwise_grad as egrad

2022-10-03 14:19:52.522874: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
function = Sphere(dimensionality=2)

In [3]:
def convert_autograd_to_tensorflow(func):#S:func is completely written in numpy autograd
    @tf.custom_gradient
    def wrapper(x):
        vjp, ans = autograd.core.make_vjp(func, x.numpy())
        def first_grad(dy):            
            @tf.custom_gradient
            def jacobian(a):
                vjp2, ans2 =  autograd.core.make_vjp(egrad(func), a.numpy())
                return ans2,vjp2 # hessian                    

            return dy* jacobian(x)  
        return ans, first_grad
    
    return wrapper

In [4]:
class Model(tf.keras.Model):

  def __init__(self, seed=None, args=None):
    super().__init__()
    self.seed = seed
    self.env = args

######################### Pixel Model Class
class PixelModel(Model):
  """
    The class for performing optimization in the input space of the functions.
    The initial parameters are chosen uniformly from [0,1] so as to ensure
        similarity across all functions
    TODO: May need to add the functionality to output denormalized bounds
  """
  def __init__(self, seed=None, args = None):
    super().__init__(seed)
    z_init = tf.random.uniform((1,args['dim']), minval = 0, maxval = 1.0)
    self.z = tf.Variable(z_init, trainable=True, dtype = tf.float32)#S:ADDED 

  def call(self, inputs=None):
    return self.z

In [5]:
model = PixelModel(None,args={'dim': 2}) # Build the model
fval = []   # To store the function values at each optimziation step   
outs = []   #Storing teh outputs of the model (normalized!!) 
tvars = model.trainable_variables

2022-10-03 14:19:55.412938: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
func = convert_autograd_to_tensorflow(function.__call__)

In [7]:
func(tf.cast(model(None), tf.float64))

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[81.98852771]])>

In [8]:
with tf.GradientTape() as tape:
    tape.watch(tvars)
    logits = 0.0 + tf.cast(model(None), tf.float64)           
    loss = func(tf.reshape(logits, (function.dimensionality)))
grad = tape.gradient(loss,tvars)
grad, loss

([<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-69.74437, 171.82608]], dtype=float32)>],
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[81.98852771]])>)

In [9]:
optimizer = tf.keras.optimizers.Adam()

In [10]:
optimizer.apply_gradients(zip(grad, tvars))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [11]:
max_iterations = 1000

In [12]:
for i in range(max_iterations + 1):
    with tf.GradientTape() as tape:
        tape.watch(tvars)
        logits = 0.0 + tf.cast(model(None), tf.float64)           
        loss = func(tf.reshape(logits, (function.dimensionality)))

    # fval.append(loss.numpy().copy())
    # outs.append(logits.numpy()[0].copy())

    grads = tape.gradient(loss, tvars)
    optimizer.apply_gradients(zip(grads, tvars))

In [13]:
tvars

[<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0.3915852 , 0.31545168]], dtype=float32)>]

In [14]:
function.get_global_minimum(2)

(array([[0.39192986, 0.17665298]]), array([[0.]]))

In [15]:
loss.numpy().copy()

array([[2.03015207]])

In [17]:
function(np.array([0.0,0.0]))

array([[19.37928748]])

In [19]:
logits.numpy().copy()

array([[0.39158124, 0.31579649]])